In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [3]:
df_pop = pd.read_csv('../data/eurostat/eu_1990_2023_population.csv', encoding='latin-1')
df_tot_area = pd.read_csv('../data/eurostat/eu_1990_2023_totalarea.csv', encoding='latin-1')
df_base = df_pop.merge(df_tot_area, how='inner', on='City', suffixes=('_pop','_area'))

In [ ]:
df_total_patents = pd.read_csv('../data/eurostat/eu_1977_2012_total_patent.csv', encoding='latin-1')
df_gva = pd.read_csv('../data/eurostat/eu_1995_2021_gva_euro_millions.csv', encoding='latin-1')
df_trademarks = pd.read_csv('../data/eurostat/eu_1996_2016_trademarks_permillion.csv', encoding='latin-1')
df_gdp_euro_current = pd.read_csv('../data/eurostat/eu_2000_2021_gdp_euro_millions_currentprices.csv', encoding='latin-1')
df_gdp_purchasingpower = pd.read_csv('../data/eurostat/eu_2000_2021_gdp_euro_millions_purchasingpower.csv', encoding='latin-1')
df_gdp_euro_perperson = pd.read_csv('../data/eurostat/eu_2000_2021_gdp_euro_perperson_currentprices.csv', encoding='latin-1')
df_gdp_purchasingpower_perperson = pd.read_csv('../data/eurostat/eu_2000_2021_gdp_euro_perperson_purchasingpower.csv', encoding='latin-1')
df_burglaries = pd.read_csv('../data/eurostat/eu_2008_2020_burglaries.csv', encoding='latin-1')
df_homicides = pd.read_csv('../data/eurostat/eu_2008_2020_homicides.csv', encoding='latin-1')
df_motortheft = pd.read_csv('../data/eurostat/eu_2008_2020_motortheft.csv', encoding='latin-1')
df_robberies = pd.read_csv('../data/eurostat/eu_2008_2020_robberies.csv', encoding='latin-1')
df_educ_lowersecondary = pd.read_csv('../data/eurostat/eu_2009_2022_lowersecondary.csv', encoding='latin-1')
df_educ_tertiary = pd.read_csv('../data/eurostat/eu_2009_2022_tertiary.csv', encoding='latin-1')
df_educ_upsecondary_nontertiary = pd.read_csv('../data/eurostat/eu_2009_2022_uppersecondary_to_nontertiary.csv', encoding='latin-1')
df_educ_upsecondary_tertiary = pd.read_csv('../data/eurostat/eu_2009_2022_uppersecondary_to_tertiary.csv', encoding='latin-1')

In [ ]:
dfs = [df_base, df_total_patents, df_gva, df_trademarks, df_gdp_euro_current, df_gdp_purchasingpower,
       df_gdp_euro_perperson, df_gdp_purchasingpower_perperson, df_burglaries, df_homicides,
       df_motortheft, df_robberies, df_educ_lowersecondary, df_educ_tertiary,
       df_educ_upsecondary_nontertiary, df_educ_upsecondary_tertiary]

for df in dfs:
    df.replace(":", np.nan, inplace=True)

In [ ]:
year = str(year)
df_new_base = df_base.filter(like=year).copy()
df_new_base['City'] = df_base['City']
df_observable = globals()[f'df_{observable}'].filter(like=year).copy()
df_observable['City'] = globals()[f'df_{observable}']['City']
df = df_new_base.merge(df_observable, how='inner', on='City')
df = df[['City', f'{year}_pop', f'{year}_area', f'{year}']].rename(columns={f'{year}_pop': 'Population', f'{year}_area': 'Area', f'{year}': 'Observable'}).apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce') if x.name in ['Population', 'Area', 'Observable'] else x).dropna()
df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda x: np.log(x))

In [ ]:
df['Interaction'] = df['Population']*df['Area']
df['Population_sq'] = df['Population']*df['Population']
df['Area_sq'] = df['Area']*df['Area']

X_col = sm.add_constant(df['Population'])
X = sm.add_constant(df[['Population','Area']])
X_p = sm.add_constant(df['Population'])
X_a = sm.add_constant(df['Area'])
X_i = sm.add_constant(df[['Population','Area','Interaction']])
X_f = sm.add_constant(df[['Population','Area','Interaction','Population_sq','Area_sq']])
y = df['Observable']

ols_col = sm.OLS(df['Area'], X_col)
ols = sm.OLS(y.values, X)
ols_p = sm.OLS(y.values, X_p)
ols_a = sm.OLS(y.values, X_a)
ols_i = sm.OLS(y.values, X_i)
ols_f = sm.OLS(y.values, X_f)
ols_result_col = ols_col.fit()
ols_result = ols.fit()
ols_result_p = ols_p.fit()
ols_result_a = ols_a.fit()
ols_result_i = ols_i.fit()
ols_result_f = ols_f.fit()

y_pred_col = ols_result_col.predict(X_col)
y_pred = ols_result.predict(X)
y_pred_p = ols_result_p.predict(X_p)
y_pred_a = ols_result_a.predict(X_a)
y_pred_i = ols_result_i.predict(X_i)
y_pred_f = ols_result_f.predict(X_f)

model = LinearRegression()
cv = RepeatedKFold(n_splits=5, n_repeats=10)
mse_scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv)
mse_scores_p = cross_val_score(model, X_p, y, scoring='neg_mean_squared_error', cv=cv)
mse_scores_a = cross_val_score(model, X_a, y, scoring='neg_mean_squared_error', cv=cv)
mse_scores_i = cross_val_score(model, X_i, y, scoring='neg_mean_squared_error', cv=cv)
mse_scores_f = cross_val_score(model, X_f, y, scoring='neg_mean_squared_error', cv=cv)
rmse_scores = np.sqrt(-mse_scores)
rmse_scores_p = np.sqrt(-mse_scores_p)
rmse_scores_a = np.sqrt(-mse_scores_a)
rmse_scores_i = np.sqrt(-mse_scores_i)
rmse_scores_f = np.sqrt(-mse_scores_f)